In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130550

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,25446.0110
2018-02-28,20915.1653
2018-03-31,24221.0691
2018-04-30,13096.3626
2018-05-31,20702.6936
2018-06-30,25551.8353
2018-07-31,29471.5419
2018-08-31,27600.2582
2018-09-30,21748.0919


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    25446.0110
 2018-02-28    20915.1653
 2018-03-31    24221.0691
 2018-04-30    13096.3626
 2018-05-31    20702.6936
 2018-06-30    25551.8353
 2018-07-31    29471.5419
 2018-08-31    27600.2582
 2018-09-30    21748.0919
 2018-10-31    21664.1119
 2018-11-30    23976.9170
 2018-12-31    31814.5172
 2019-01-31    36594.2409
 2019-02-28    29427.7197
 2019-03-31    32992.8814
 2019-04-30    29592.3379
 2019-05-31    24482.6153
 2019-06-30    37595.0668
 2019-07-31    35539.4860
 2019-08-31    29765.0631
 2019-09-30    26641.7941
 2019-10-31    28501.5210
 2019-11-30    26657.8691
 2019-12-31    28196.4711
 2020-01-31    25720.4375
 2020-02-29    22386.3738
 2020-03-31    25211.4122
 2020-04-30    30073.6061
 2020-05-31    35683.4260
 2020-06-30    24357.1555
 2020-07-31    37328.2809
 2020-08-31    33434.5764
 2020-09-30    26069.3192
 2020-10-31    31439.1760
 2020-11-30    34803.5620
 2020-12-31    27342.8881
 2021-01-31    29446.9431
 2021-02-

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.026192
p-value : 0.032520
Critical Values :
	1%: -3.646135
	5%: -2.954127
	10%: -2.615968


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=841.872, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=839.476, Time=0.12 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=841.710, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=844.627, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=847.023, Time=0.09 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.36 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=838.260, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=840.687, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=842.537, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=845.433, Time=0.10 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=847.345, Time=0.11 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=840.120, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=842.643, Time=0.08 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=844.291, Time=0.19 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -416.125
Method:                       css-mle   S.D. of innovations           4852.358
Date:                Sat, 11 Sep 2021   AIC                            838.251
Time:                        04:40:15   BIC                            843.464
Sample:                             0   HQIC                           840.161
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.754e+04   1144.184     24.073      0.000    2.53e+04    2.98e+04
ar.L1.y        0.3533      0.143      2.463      0.014       0.072       0.634
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([26061.22770488, 27020.21152949]), array([4852.35818006, 5146.25722943]), array([[16550.78043187, 35571.6749779 ],
       [16933.73270464, 37106.69035435]]))
